# Introduction
I wanted to learn SQL by using Python, so I searched for a SQL cheatsheet that I could practice with Python. I came across datacamp’s SQL basics cheat sheet [link](https://www.datacamp.com/cheat-sheet/sql-basics-cheat-sheet), which looked useful. However, it did not provide any sample data to work with. To make the most of this document, I decided to create a database that had all the columns shown in the cheat sheet's example from a spreadsheet [Listings.csv](https://www.kaggle.com/datasets/mysarahmadbhat/airbnb-listings-reviews?resource=download). This practice felt like data cleanup activity.

# Cleanup process
Listings.csv has many columns, but the database has id, city, country, number_of_rooms and year_listed columns. Since this, cleanup activities shhould be done:
1. Cloning new columns from existing columns
2. Removing unused columns
3. Adding new columns which are based on existing columns
4. Removing rows which have NA (Not Applicable) values
5. Making sure that type of some columns are correct one

Two first lines import pandas and read data from csv file to dataframe. Next steps, cloning new columns 'id','number_of_rooms', 'year_listed' from 'listing_id', 'bed_rooms', 'host_since' columns respectively. Last line removed all unused columns from dataframe by using function [<Pandas.DataFrame>.drop()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html). axis='columns: remove columns, inplace=True: removing directly to dataframe itself.

In [1]:
import pandas as pd
import sqlite3
# adding encoding = "ISO-8859-1" due to there is an error once converting csv to dataframe
df = pd.read_csv("Data/Listings.csv",encoding = "ISO-8859-1")
# create new columns which are cloned from existing columns
df['id'] = df['listing_id']
df['number_of_rooms'] = df['bedrooms']
df['year_listed'] = df['host_since']
# delete unused columns
df.drop(['name', 'host_id', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'district', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin',  'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'listing_id','host_location', 'bedrooms', 'host_since'],axis='columns', inplace=True)

C:\Users\Cuong\AppData\Local\Temp\ipykernel_26516\1203861301.py:4: DtypeWarning: Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/Listings.csv",encoding = "ISO-8859-1")


A new column named country is added and assigned the value that corresponds to the city column. For example, if the city is New York, the country is USA. We do this by following these steps:
- A list of all the different city names in the dataset. 
- A dictionary that has the city names as keys and the country names as values is created. 
- The [Series.map()](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html) function is used to map the dictionary values to the new country column based on the keys.

In [2]:
# get all unique cities in spreadsheet
print(df['city'].unique())
# create a dictionary which has keys are cities and values are country
city_dict = {'Paris': 'France', 'New York': 'USA', 'Bangkok': 'Thailand', 'Rio de Janeiro':'Brazil', 'Sydney': 'Autralia', 'Istanbul': 'Turkey', 'Rome':'Italia', 'Hong Kong': 'Hong Kong', 'Mexico City': 'Mexico', 'Cape Town': 'South Africa'}
df['country']=df['city'].map(city_dict)

['Paris' 'New York' 'Bangkok' 'Rio de Janeiro' 'Sydney' 'Istanbul' 'Rome'
 'Hong Kong' 'Mexico City' 'Cape Town']


The format of a column named year_listed is changed from YYYY-MM_DD to YYYY (for example, from 2011-12-03 to 2011) by splitting the string with ‘-’ and taking the first element. This can be done by using the function [Series.str.split](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html) with pat=‘-’ as an argument. Then, the first element in each row of “year_listed” is selected.

In [3]:
df['year_listed']=df['year_listed'].str.split(pat='-')
df['year_listed']=df['year_listed'].str[0]

Any row with a NA value is deleted by [df.dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html). The existing dataframe is modified directly by setting inplace=True as an argument.

In [4]:
df.dropna(inplace=True)

Next three lines of code converts the types of each column to the correct ones, such as integer type for year_listed, number_of_rooms, and id columns. It does this by using astype(int) on each column. 

In [5]:
df['year_listed']=df['year_listed'].astype(int)
df['number_of_rooms']=df['number_of_rooms'].astype(int)
df['id']=df['id'].astype(int)

Tip: Comparing a large dataframe before and after using the dropna function can be hard. But the next 2 lines can display all the rows that were removed.

In [6]:
# Two below lines print NA rows which were deleted.
df_after= df.dropna()
print(pd.concat([df,df_after]).drop_duplicates(keep=False))

Empty DataFrame
Columns: [city, id, number_of_rooms, year_listed, country]
Index: []


# Export datafrae to SQLite database
In the end, the dataframe has all the required columns. A SQLite database can be created by using [to_sql](https://medium.com/@pc0is0me/turn-single-spreadsheet-into-a-sqlite-database-a70069787020) function.  

In [7]:
with sqlite3.connect('airbnb_listings.db') as con:
    df.to_sql('airbnb_listings', con=con, if_exists='append',index=False)

# Conclusion:
 This post ends here after the SQLite database is created. By following all the steps above, I hope you can learn some ways to clean a dataframe.